# IMPORTACIÓN DE LIBRERIAS

In [3]:
import pandas as pd
import time
import smtplib
from email.mime.application import MIMEApplication
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import math  # Necesario para verificar NaN
import os
import pywhatkit
import time
import pyautogui

# EXTRACCIÓN O LLAMADO DE MI DATA FRAME

In [4]:
# Leer el registro desde un archivo Excel
data_df = pd.read_excel(r'C:\Users\jalopezz\Desktop\Pruebas_Excels\MADRUGA.xlsx', dtype={"Numero": str})

# Convertir el archivo Excel en un DataFrame de Pandas
info_backlog = pd.DataFrame(data_df)

# SELECCIONANDO COLUMNAS DE MI DATA FRAME

In [5]:
# Seleccionar columnas por nombre
info_backlog_ = info_backlog[['deliveryOrderNumber', 'lastStatusGSC', 'seller_Name', 'sku', 'description', 'segmento', 'promisedBySeller', 'accion', 'Correo', 'Numero', 'loyaltyPhone', 'Speech']]


# AGREGANDO LOS FILTROS Y/O CONDICIONES 

In [6]:
# Filtrar registros AAA que cumplen con ciertas condiciones
info_backlog_1 = info_backlog[(info_backlog['segmento'] == 'AAA') & 
                              (~info_backlog['accion'].isin(['No accionar', 'Cancelar', 'Cambio de estado']))]
print(info_backlog_1)

# Filtrar registros para Speech_2 que cumplen con ciertas condiciones
info_backlog_2 = info_backlog[(info_backlog['segmento'] != 'AAA') & 
                              (info_backlog['Speech'] == 'Speech_2') & 
                              (~info_backlog['accion'].isin(['No accionar', 'Cancelar', 'Cambio de estado']))]
print(info_backlog_2)

# Filtrar registros para Speech_3 que cumplen con ciertas condiciones
info_backlog_3 = info_backlog[(info_backlog['segmento'] != 'AAA') & 
                              (info_backlog['Speech'] == 'Speech_3') & 
                              (~info_backlog['accion'].isin(['No accionar', 'Cancelar', 'Cambio de estado']))]
print(info_backlog_3)

# Filtrar registros para Speech_4 que cumplen con ciertas condiciones
info_backlog_4 = info_backlog[(info_backlog['segmento'] != 'AAA') & 
                              (info_backlog['Speech'] == 'Speech_4') & 
                              (~info_backlog['accion'].isin(['No accionar', 'Cancelar', 'Cambio de estado']))]
print(info_backlog_4)

          datecreatedAt  deliveryOrderNumber        trackingCode  \
19  2024-02-19T21:39:35           2168198601  240121000001652992   
21  2024-02-19T22:53:14           2168223251  240111000001652992   
22  2024-02-20T08:47:25           2168347509  240111000001654016   
23  2024-02-19T20:14:48           2168163523  240121000001652000   
24  2024-02-20T11:53:52           2168415643  240121000001656000   
25  2024-02-17T22:26:14           2167391872  240121000001640000   
26  2024-02-19T20:47:02           2168177733  240111000001652000   
27  2024-02-11T01:19:19           2164604220  240121000001584992   
28  2024-02-20T10:26:21           2168381681  240111000001655008   
29  2024-02-19T20:47:02           2168177733  240111000001652000   
30  2024-02-20T10:26:21           2168381681  240111000001655008   
31  2024-02-19T20:47:02           2168177733  240111000001652000   
32  2024-02-19T20:47:02           2168177733  240111000001652000   
33  2024-02-20T11:20:37           2168402402  24

# BLOQUE PARA ENVIAR MAILS A AAA

In [7]:

correo_usuario = 'aoriveram@falabella.com'
contrasena = 'Marinette2023.'
asunto_base = 'F.COM | 🚨 HOY ESTAS EN RUTA🚨'

# Establecer la conexión con el servidor SMTP de Outlook
server = smtplib.SMTP('smtp.office365.com', 587)
server.starttls()

# Iniciar sesión en el servidor SMTP
server.login(correo_usuario, contrasena)

# Iterar sobre las tiendas únicas en el DataFrame
for seller in info_backlog_1['seller_ID'].unique():
    # Filtrar el DataFrame para la tienda actual
    df_seller = info_backlog_1[info_backlog_1['seller_ID'] == seller]


    # Obtener la dirección de correo electrónico de la tienda actual
    correo_seller = df_seller['Correo'].iloc[0]
    # Obtener el número de teléfono asignado al vendedor actual
    phone_asignado = df_seller['loyaltyPhone'].iloc[0]

    # Verificar si la dirección de correo electrónico es NaN
    if pd.isna(correo_seller):
        print(f"La dirección de correo electrónico para la tienda {seller} es nula (NaN). No se enviará el correo.")
        continue  # Saltar a la siguiente iteración

    # Obtener el valor de seller_Name para la tienda actual
    seller_name = df_seller['seller_Name'].iloc[0]

    # Seleccionar columnas específicas para mostrar en el cuerpo del correo
    columnas_seleccionadas = ['seller_Name', 'deliveryOrderNumber', 'sku', 'description', 'promisedBySeller', 'accion']
    df_seller_seleccionada = df_seller[columnas_seleccionadas].drop_duplicates()
    df_seller_seleccionada  =  df_seller_seleccionada .sort_values(by='accion', ascending=False)
    
    # Configurar el mensaje
    msg = MIMEMultipart()
    msg['From'] = 'fcomlogisticasx@falabella.com'
    msg['To'] = correo_seller
    msg['Subject'] = asunto_base.format(seller)
    msg['CC'] = 'jalopezz@Falabella.com'

    # Crear el cuerpo del correo en formato HTML
    mensaje_html = f"""
<html>
  <head>
    <style>
      table {{
        border-collapse: collapse;
        width: 100%;
      }}
      th, td {{
        border: 1px solid black;
        padding: 8px;
        text-align: left;
      }}
      th {{
        background-color: orange;
        color: white;
        font-size: 16px;
        font-weight: bold;
        text-align: center;
      }}
    </style>
  </head>
  <body>
    <p><h2><strong>Hola {seller_name}, con ID {seller}!</strong> estas ordenes son las que te recolectaremos hoy </h2></p></p>
    
    <br>
    {df_seller_seleccionada.to_html(index=False)}
    <br>
    <p>Queremos reforzar contigo el proceso para gestionarlas:</p>
        <p>📦 Para que podamos recolectar tu orden deberás:</p>
        <ol>
            <li>Revisar tus órdenes <a href="https://sellercenter.falabella.com/order">aquí</a>.</li>
            <li>Imprimir la <em>etiqueta</em> que pegarás al exterior de tu empaque. Se visualiza al lado izquierdo como <em>Label</em>.</li>
            <li>Hacer clic en <em>Listo para despachar</em> - máximo un día antes de la fecha límite (antes de las 8:00PM).</li>
        </ol>
        <p>🚚 Revisa nuestro tutorial de despacho: <a href="https://youtu.be/lpk1_y9uYDg">Tutorial de despacho</a>.</p>
        <p>Siguiendo estos pasos estaremos programando la recolección para el día siguiente.</p>


        <p>En caso desees un soporte más personalizado escríbenos al <strong>{phone_asignado}</strong>.</p>
        <p>¡Gracias!</p> 
  </body>
</html>
"""

    # Adjuntar el cuerpo del correo en formato HTML
    msg.attach(MIMEText(mensaje_html, 'html'))

    # Enviar el correo electrónico
    server.sendmail(correo_usuario, correo_seller, msg.as_string())

# Cerrar la conexión con el servidor SMTP
server.quit()



(221, b'2.0.0 Service closing transmission channel')

# BLOQUE PARA ENVIAR MAILS A OTHERS SELLERS (MAÑANA ESTAS EN RUTA Y RETRASO) & PENDING

In [8]:

# Convertir todas las columnas a cadenas para evitar el error de 'float' object has no attribute 'encode'
info_backlog_2= info_backlog_2.applymap(lambda x: str(x))

correo_usuario = 'aoriveram@falabella.com'
contrasena = 'Marinette2023.'
asunto_base = 'F.COM | 🚨 HOY ESTAS EN RUTA🚨'

# Establecer la conexión con el servidor SMTP de Outlook
server = smtplib.SMTP('smtp.office365.com', 587)
server.starttls()

# Iniciar sesión en el servidor SMTP
server.login(correo_usuario, contrasena)


# Iterar sobre las tiendas únicas en el DataFrame
for seller in info_backlog_2['seller_ID'].unique():
    # Filtrar el DataFrame para la tienda actual
    df_seller_1 = info_backlog_2[info_backlog_2['seller_ID'] == seller]

    # Obtener la dirección de correo electrónico de la tienda actual
    correo_seller = df_seller_1['Correo'].iloc[0]

    # Verificar si la dirección de correo electrónico es NaN
    if isinstance(correo_seller, float) and math.isnan(correo_seller):
        print(f"La dirección de correo electrónico para la tienda {seller} es nula (NaN). No se enviará el correo.")
        continue  # Saltar a la siguiente iteración
   # Obtener el valor de seller_ID para la tienda actual
    seller_name = df_seller_1['seller_Name'].iloc[0]

    # Seleccionar columnas específicas para mostrar en el cuerpo del correo
    columnas_seleccionadas_2 = ['seller_Name', 'deliveryOrderNumber', 'sku', 'description', 'promisedBySeller', 'accion']
    df_seller_seleccionada_2 = df_seller_1[columnas_seleccionadas_2].drop_duplicates()
    
    # Ordenar el DataFrame por la columna 'accion' de manera descendente
    df_seller_seleccionada_2 = df_seller_seleccionada_2.sort_values(by='accion', ascending=False)

    # Configurar el mensaje
    msg = MIMEMultipart()
    msg['From'] = 'fcomlogisticasx@falabella.com'
    msg['To'] = correo_seller
    msg['Subject'] = asunto_base.format(seller)
    msg['CC'] = 'jalopezz@Falabella.com'

    # Crear el cuerpo del correo en formato HTML
    mensaje_html = f"""
<html>
  <head>
    <style>
      table {{
        border-collapse: collapse;
        width: 100%;
      }}
      th, td {{
        border: 1px solid black;
        padding: 8px;
        text-align: left;
      }}
      th {{
        background-color: orange;
        color: white;
        font-size: 16px;
        font-weight: bold;
        text-align: center;
      }}
    </style>
  </head>
   <body>
    <p><h2><strong>Hola {seller_name}, con ID {seller}!</strong> estas ordenes son las que te pasaremos a recolectar el día de hoy </h2></p></p>
    
    <br>
    {df_seller_seleccionada_2.to_html(index=False)}
    <br>
    <p>*Tener en cuenta que si tienes ordenes con días de retraso ya están impactando el cumplimiento de tus indicadores. Recuerda que tu nivel de cumplimiento se relaciona directamente con la cantidad cobrada por cofinanciamiento logístico. Si necesitas más detalles sobre estos cobros, haz clic aqui :<a href="https://www.falabellasellers.com/comisiones-y-costos">https://www.falabellasellers.com/comisiones-y-costos</a></li></p>
    <p>*Las ordenes con 3 días de retraso de no ser entregadas será cancelada el día de hoy.</p>
    <br>


        <p><h2>🚨 Finalmente, detectamos que tienes <strong>órdenes pendientes</strong>  por cambiar de estado 🚨</h2></p>
        <p>Recuerda que debes cambiar el estado de estas órdenes a Listo para despachar un día previo de la fecha límite de despacho, como máximo hasta las 10 pm, para que podamos incluirlas en la ruta de recolección</p>
        <p>Queremos reforzar contigo el proceso para gestionarlas lo más pronto posible y evitar cancelaciones:</p>
        <ol>
            <li>Revisar tus órdenes aquí: <a href="https://sellercenter.falabella.com/order">https://sellercenter.falabella.com/order</a></li>
            <li>Imprimir la <em>etiqueta</em> que pegarás al exterior de tu empaque. Se visualiza al lado izquierdo como <em>Label</em>.</li>
            <li>Hacer clic en <em>Listo para despachar</em> - máximo un día antes de la fecha límite de despacho (antes de las 8:00PM).</li>
        </ol>
        <p>🚚 Revisa nuestro tutorial de despacho: <a href="https://youtu.be/lpk1_y9uYDg">Tutorial de despacho</a>.</p>
        <p>Siguiendo estos pasos estaremos programando la recolección para el día siguiente.</p>
        <p>¡No pierdas ventas! Recuerda que si la orden se cancela no habrá lugar a reclamo.</p>

        <p>*Recuerde: que no es necesario el envío de este correo para la recolección de estas órdenes. Asimismo, la unidad tiene un tiempo de espera de atención de 10 minutos. De marcarse el fallo y ser recurrente, se procede a evaluar ciertas penalidades sin opción a reclamo.</p>
        
        <p>¡Contamos con tu apoyo!</p>
        
  </body>
</html>
"""

      # Adjuntar el cuerpo del correo en formato HTML
    msg.attach(MIMEText(mensaje_html, 'html'))

    # Enviar el correo electrónico
    server.sendmail(correo_usuario, msg['To'], msg.as_string())

# Cerrar la conexión con el servidor SMTP
server.quit()


(221, b'2.0.0 Service closing transmission channel')

# BLOQUE PARA ENVIAR MAILS A OTHERS SELLERS (MAÑANA ESTAS EN RUTA Y RETRASO)

In [9]:

# Convertir todas las columnas a cadenas para evitar el error de 'float' object has no attribute 'encode'
info_backlog_3= info_backlog_3.applymap(lambda x: str(x))

correo_usuario = 'aoriveram@falabella.com'
contrasena = 'Marinette2023.'
asunto_base = 'F.COM | 🚨 HOY ESTAS EN RUTA🚨'

# Establecer la conexión con el servidor SMTP de Outlook
server = smtplib.SMTP('smtp.office365.com', 587)
server.starttls()

# Iniciar sesión en el servidor SMTP
server.login(correo_usuario, contrasena)


# Iterar sobre las tiendas únicas en el DataFrame
for seller in info_backlog_3['seller_ID'].unique():
    # Filtrar el DataFrame para la tienda actual
    df_seller_2 = info_backlog_3[info_backlog_3['seller_ID'] == seller]

    # Obtener la dirección de correo electrónico de la tienda actual
    correo_seller = df_seller_2['Correo'].iloc[0]

    # Verificar si la dirección de correo electrónico es NaN
    if isinstance(correo_seller, float) and math.isnan(correo_seller):
        print(f"La dirección de correo electrónico para la tienda {seller} es nula (NaN). No se enviará el correo.")
        continue  # Saltar a la siguiente iteración
   # Obtener el valor de seller_ID para la tienda actual
    seller_name = df_seller_2['seller_Name'].iloc[0]

    # Seleccionar columnas específicas para mostrar en el cuerpo del correo
    columnas_seleccionadas_3 = ['seller_Name', 'deliveryOrderNumber', 'sku', 'description', 'promisedBySeller', 'accion']
    df_seller_seleccionada_3 = df_seller_2[columnas_seleccionadas_3].drop_duplicates()
    
    # Ordenar el DataFrame por la columna 'accion' de manera descendente
    df_seller_seleccionada_3 = df_seller_seleccionada_3.sort_values(by='accion', ascending=False)

    # Configurar el mensaje
    msg = MIMEMultipart()
    msg['From'] = 'fcomlogisticasx@falabella.com'
    msg['To'] = correo_seller
    msg['Subject'] = asunto_base.format(seller)
    msg['CC'] = 'jalopezz@Falabella.com'

    # Crear el cuerpo del correo en formato HTML
    mensaje_html = f"""
<html>
  <head>
    <style>
      table {{
        border-collapse: collapse;
        width: 100%;
      }}
      th, td {{
        border: 1px solid black;
        padding: 8px;
        text-align: left;
      }}
      th {{
        background-color: orange;
        color: white;
        font-size: 16px;
        font-weight: bold;
        text-align: center;
      }}
    </style>
  </head>
   <body>
    <p><h2><strong>Hola {seller_name}, con ID {seller}!</strong> estas ordenes son las que te pasaremos a recolectar el día de hoy </h2></p></p>
    
    <br>
    {df_seller_seleccionada_3.to_html(index=False)}
    <br>
    <p>*Tener en cuenta que si tienes ordenes con días de retraso ya están impactando el cumplimiento de tus indicadores. Recuerda que tu nivel de cumplimiento se relaciona directamente con la cantidad cobrada por cofinanciamiento logístico. Si necesitas más detalles sobre estos cobros, haz clic aqui :<a href="https://www.falabellasellers.com/comisiones-y-costos">https://www.falabellasellers.com/comisiones-y-costos</a></li></p>
    <p>*Las ordenes con 3 días de retraso de no ser entregadas será cancelada el día de hoy.</p>

        <p>*Recuerde: que no es necesario el envío de este correo para la recolección de estas órdenes. Asimismo, la unidad tiene un tiempo de espera de atención de 10 minutos. De marcarse el fallo y ser recurrente, se procede a evaluar ciertas penalidades sin opción a reclamo.</p>
        
        <p>¡Contamos con tu apoyo!</p>
        
  </body>
</html>
"""

      # Adjuntar el cuerpo del correo en formato HTML
    msg.attach(MIMEText(mensaje_html, 'html'))

    # Enviar el correo electrónico
    server.sendmail(correo_usuario, msg['To'], msg.as_string())

# Cerrar la conexión con el servidor SMTP
server.quit()


(221, b'2.0.0 Service closing transmission channel')

# BLOQUE PARA ENVIAR MAILS ONLY PENDINGS

In [10]:
# Convertir todas las columnas a cadenas para evitar el error de 'float' object has no attribute 'encode'
info_backlog_4= info_backlog_4.applymap(lambda x: str(x))

correo_usuario = 'aoriveram@falabella.com'
contrasena = 'Marinette2023.'
asunto_base = 'F.COM | 🚨 REALIZA EL CAMBIO DE ESTADO DE TUS ORDENES🚨'

# Establecer la conexión con el servidor SMTP de Outlook
server = smtplib.SMTP('smtp.office365.com', 587)
server.starttls()

# Iniciar sesión en el servidor SMTP
server.login(correo_usuario, contrasena)


# Iterar sobre las tiendas únicas en el DataFrame
for seller in info_backlog_4['seller_ID'].unique():
    # Filtrar el DataFrame para la tienda actual
    df_seller_3 = info_backlog_4[info_backlog_4['seller_ID'] == seller]

    # Obtener la dirección de correo electrónico de la tienda actual
    correo_seller = df_seller_3['Correo'].iloc[0]

    # Verificar si la dirección de correo electrónico es NaN
    if isinstance(correo_seller, float) and math.isnan(correo_seller):
        print(f"La dirección de correo electrónico para la tienda {seller} es nula (NaN). No se enviará el correo.")
        continue  # Saltar a la siguiente iteración
   # Obtener el valor de seller_ID para la tienda actual
    seller_name = df_seller_3['seller_Name'].iloc[0]

    # Seleccionar columnas específicas para mostrar en el cuerpo del correo
    columnas_seleccionadas_4 = ['seller_Name', 'deliveryOrderNumber', 'sku', 'description', 'promisedBySeller', 'accion']
    df_seller_seleccionada_4 = df_seller_3[columnas_seleccionadas_4].drop_duplicates()
    
    # Ordenar el DataFrame por la columna 'accion' de manera descendente
    df_seller_seleccionada_4 = df_seller_seleccionada_4.sort_values(by='accion', ascending=False)

    # Configurar el mensaje
    msg = MIMEMultipart()
    msg['From'] = 'fcomlogisticasx@falabella.com'
    msg['To'] = correo_seller
    msg['Subject'] = asunto_base.format(seller)
    msg['CC'] = 'jalopezz@Falabella.com'

    # Crear el cuerpo del correo en formato HTML
    mensaje_html = f"""
<html>
  <head>
    <style>
      table {{
        border-collapse: collapse;
        width: 100%;
      }}
      th, td {{
        border: 1px solid black;
        padding: 8px;
        text-align: left;
      }}
      th {{
        background-color: orange;
        color: white;
        font-size: 16px;
        font-weight: bold;
        text-align: center;
      }}
    </style>
  </head>
   <body>
    <p><h2><strong>Hola {seller_name}, con ID {seller}!</strong> no haz realizado el cambio de estado de las siguientes ordenes </h2></p></p>
    
    <br>
    {df_seller_seleccionada_4.to_html(index=False)}
    <br>
    <p>*Tener en cuenta que si tienes ordenes con días de retraso ya están impactando el cumplimiento de tus indicadores. Recuerda que tu nivel de cumplimiento se relaciona directamente con la cantidad cobrada por cofinanciamiento logístico. Si necesitas más detalles sobre estos cobros, haz clic aqui :<a href="https://www.falabellasellers.com/comisiones-y-costos">https://www.falabellasellers.com/comisiones-y-costos</a></li></p>
    <p>*Las ordenes con 3 días de retraso de no ser entregadas será cancelada el día de hoy.</p>

    <p>Recuerda que debes cambiar el estado de estas órdenes a Listo para despachar un día previo de la fecha límite de despacho, como máximo hasta las 10 pm, para que podamos incluirlas en la ruta de recolección</p>
    <p>Queremos reforzar contigo el proceso para gestionarlas lo más pronto posible y evitar cancelaciones:</p>
    <ol>
        <li>Revisar tus órdenes aquí: <a href="https://sellercenter.falabella.com/order">https://sellercenter.falabella.com/order</a></li>
        <li>Imprimir la <em>etiqueta</em> que pegarás al exterior de tu empaque. Se visualiza al lado izquierdo como <em>Label</em>.</li>
        <li>Hacer clic en <em>Listo para despachar</em> - máximo un día antes de la fecha límite de despacho (antes de las 8:00PM).</li>
    </ol>
    <p>🚚 Revisa nuestro tutorial de despacho: <a href="https://youtu.be/lpk1_y9uYDg">Tutorial de despacho</a>.</p>
    <p>Siguiendo estos pasos estaremos programando la recolección para el día siguiente.</p>
    <p>¡No pierdas ventas! Recuerda que si la orden se cancela no habrá lugar a reclamo.</p>

    <p>*Recuerde: que no es necesario el envío de este correo para la recolección de estas órdenes. Asimismo, la unidad tiene un tiempo de espera de atención de 10 minutos. De marcarse el fallo y ser recurrente, se procede a evaluar ciertas penalidades sin opción a reclamo.</p>
    
    <p>¡Contamos con tu apoyo!</p>
        
  </body>
</html>
"""

      # Adjuntar el cuerpo del correo en formato HTML
    msg.attach(MIMEText(mensaje_html, 'html'))

    # Enviar el correo electrónico
    server.sendmail(correo_usuario, msg['To'], msg.as_string())

# Cerrar la conexión con el servidor SMTP
server.quit()


(221, b'2.0.0 Service closing transmission channel')

# BLOQUE PARA ENVIAR MENSAJE DE WHATS APP A SELLERS AAA

In [11]:
mensaje_whatsapp_1 = """
*Hola {seller_Name}, tienes órdenes pendientes de envío ⚠️*
Para mayor detalle te hemos enviado un mail al correo *{correo}* con asunto "F.COM | 🚨 HOY ESTAS EN RUTA🚨"

Queremos reforzar contigo el proceso para gestionarlas:

📦 Para que podamos recolectar tu orden deberás:

1- Revisar tus órdenes aquí: https://sellercenter.falabella.com/order

2- Imprimir la *etiqueta* que pegarás al exterior de tu empaque. Se visualiza al lado izquierdo como *Label*.

3- Hacer clic en *Listo para despachar* - máximo un día antes de la fecha límite (antes de las 8:00PM).

🚚 Revisa nuestro tutorial de despacho: https://youtu.be/lpk1_y9uYDg

Siguiendo estos pasos estaremos programando la recolección para el día siguiente

En caso desees un soporte más personalizado escríbenos al *{Phone}*

¡Gracias!

"""
for seller in info_backlog_1['seller_ID'].unique():
    # Filtrar el DataFrame para la tienda actual
    df_seller = info_backlog_1[info_backlog_1['seller_ID'] == seller]
    
    
    whatsapp_message = mensaje_whatsapp_1.format(
                    seller_Name=df_seller["seller_Name"].iloc[0],
                        correo=df_seller["Correo"].iloc[0],
                        Phone =df_seller["loyaltyPhone"].iloc[0],

            )
    # Enviar el mensaje de WhatsApp
    pywhatkit.sendwhatmsg_instantly(info_backlog_1["Numero"].iloc[0], whatsapp_message)
    time.sleep(12)     


# BLOQUE PARA ENVIAR MENSAJE DE WHATS APP A OTHERS SELLERS (MAÑANA ESTAS EN RUTA Y RETRASO) & PENDING

In [12]:
mensaje_whatsapp_2 = """
*¡Hola! *{seller_Name}*!  

🚨 Detectamos que tienes órdenes pendientes de cambio de estado 🚨

Queremos reforzar contigo el proceso para gestionarlas lo más pronto posible y evitar cancelaciones:     

📦 Para que podamos recolectar tu orden deberás:      


1- Revisar tus órdenes aquí: https://sellercenter.falabella.com/order    
2- Imprimir la *etiqueta* que pegarás al exterior de tu empaque. Se visualiza al lado izquierdo como *Label*. 
3- Hacer clic en *Listo para despachar* - máximo un día antes de la fecha límite de despacho (antes de las 8:00PM).  

🚚 Revisa nuestro tutorial de despacho: https://youtu.be/lpk1_y9uYDg 

Siguiendo estos pasos estaremos programando la recolección para el día siguiente 

¡No pierdas ventas!  Recuerda que si la orden se cancela no habrá lugar a reclamo.  

⭐️ Además, te recordamos que:  

Mañana pasaremos a recolectar los pedidos programados que tengan ese día como fecha máxima de despacho: 

📋 Verifica tu lista de pedidos para mañana y tenlos a la mano para entregárselos al operador logístico.  
🚨 Importante: Solo recolectaremos las órdenes programadas, es decir aquellas que estén en estado Listo para despachar. 

Para mayor detalle te hemos enviado un mail al correo *{correo}* con asunto "F.COM | 🚨 HOY ESTAS EN RUTA🚨"

¡Contamos con tu apoyo!
"""

for seller in info_backlog_2['seller_ID'].unique():
    # Filtrar el DataFrame para la tienda actual
    df_seller_1 = info_backlog_2[info_backlog_2['seller_ID'] == seller]
    
    
    whatsapp_message_2 = mensaje_whatsapp_2.format(
                    seller_Name=df_seller_1["seller_Name"].iloc[0],
                        correo=df_seller_1["Correo"].iloc[0],
                        Phone =df_seller_1["loyaltyPhone"].iloc[0],

            )
    # Enviar el mensaje de WhatsApp
    pywhatkit.sendwhatmsg_instantly(info_backlog_2["Numero"].iloc[0], whatsapp_message_2)
    time.sleep(12)  

# BLOQUE PARA ENVIAR MENSAJE DE WHATS APP A OTHERS SELLERS (MAÑANA ESTAS EN RUTA Y RETRASO)

In [13]:
mensaje_whatsapp_3 = """
*¡Hola! *{seller_Name}*! 

🚨 Hoy estás en Ruta 🚨 

Para mayor detalle te hemos enviado un mail al correo *{correo}* con asunto "F.COM | 🚨 HOY ESTAS EN RUTA🚨"

Revisa tus órdenes pendientes aquí: https://sellercenter.falabella.com/order    

y gestiónalas a la brevedad para evitar cancelaciones de cliente por retrasos.  

¡No pierdas ventas! Cumple con los tiempos estipulados en la plataforma ya que si la orden se cancela no habrá lugar a reclamo. 


⭐️ Además, te recordamos que:  

Hoy pasaremos a recolectar los pedidos programados que tengan ese día como fecha máxima de despacho.

📋 Verifica tu lista de pedidos para hoy y tenlos a la mano para entregárselos al operador logístico.  

 🚨 Importante: Solo recolectaremos las órdenes programadas, es decir aquellas que estén en estado Listo para despachar. 

 
¡Contamos con tu apoyo!"""

for seller in info_backlog_3['seller_ID'].unique():
    # Filtrar el DataFrame para la tienda actual
    df_seller_2 = info_backlog_3[info_backlog_3['seller_ID'] == seller]
    
    
    whatsapp_message_3 = mensaje_whatsapp_3.format(
                    seller_Name=df_seller_2["seller_Name"].iloc[0],
                        correo=df_seller_2["Correo"].iloc[0],
                        Phone =df_seller_2["loyaltyPhone"].iloc[0],

            )
    # Enviar el mensaje de WhatsApp
    pywhatkit.sendwhatmsg_instantly(info_backlog_3["Numero"].iloc[0], whatsapp_message_3)
    time.sleep(12)  

# BLOQUE PARA ENVIAR MENSAJE DE WHATS APP OTHERS Y PENDING

In [14]:
mensaje_whatsapp_4 = """
*Hola {seller_Name}, tienes órdenes pendientes de envío⚠️*

Queremos reforzar contigo el proceso para gestionarlas:

📦 Para que podamos recolectar tu orden deberás:

1- Revisar tus órdenes aquí: https://sellercenter.falabella.com/order

2- Imprimir la *etiqueta* que pegarás al exterior de tu empaque. Se visualiza al lado izquierdo como *Label*.

3- Hacer clic en *Listo para despachar* - máximo un día antes de la fecha límite de despacho (antes de las 8:00PM).

🚚 Revisa nuestro tutorial de despacho: https://youtu.be/lpk1_y9uYDg

Siguiendo estos pasos estaremos programando la recolección para el día siguiente

¡Gracias!

 
¡Contamos con tu apoyo!"""

for seller in info_backlog_4['seller_ID'].unique():
    # Filtrar el DataFrame para la tienda actual
    df_seller_3 = info_backlog_4[info_backlog_4['seller_ID'] == seller]
    
    
    whatsapp_message_4 = mensaje_whatsapp_4.format(
                    seller_Name=df_seller_3["seller_Name"].iloc[0],
                        correo=df_seller_3["Correo"].iloc[0],
                        Phone =df_seller_3["loyaltyPhone"].iloc[0],

            )
    # Enviar el mensaje de WhatsApp
    pywhatkit.sendwhatmsg_instantly(info_backlog_4["Numero"].iloc[0], whatsapp_message_4)
    time.sleep(12)  